In [1]:
from pathlib import Path

import pandas as pd
from pyproj import CRS

from bedrock.gi.ags.read import ags_to_dfs
from bedrock.gi.ags.schemas import (
    Ags3CORE,
    Ags3GEOL,
    Ags3HOLE,
    Ags3ISPT,
    Ags3WETH,
    BaseSAMP,
)
from bedrock.gi.ags.transform import (
    ags3_hole_to_brgi_location,
    ags3_in_situ_to_brgi_in_situ,
    ags3_samp_to_brgi_sample,
    ags3_to_brgi,
    ags_proj_to_brgi_project,
    generate_sample_ids_for_ags3,
)
from bedrock.gi.ags.validate import check_ags3_hole_ids_exist

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [2]:
cwd = Path.cwd()
gi_dir = cwd / "gi-data" / "kaitak"
gi_files = [
    gi_dir / "31241" / "GE9908.7.ags",
    gi_dir / "44751" / "GE-2005-03-57 rev0.ags",
    gi_dir / "47615" / "GE-2007-13-4 rev0.ags",
]
gi_files

[WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/31241/GE9908.7.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/44751/GE-2005-03-57 rev0.ags'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/sandbox/gi-data/kaitak/47615/GE-2007-13-4 rev0.ags')]

In [3]:
crs = CRS(2326)
crs

<Projected CRS: EPSG:2326>
Name: Hong Kong 1980 Grid System
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: China - Hong Kong - onshore and offshore.
- bounds: (113.76, 22.13, 114.51, 22.58)
Coordinate Operation:
- name: Hong Kong 1980 Grid
- method: Transverse Mercator
Datum: Hong Kong 1980
- Ellipsoid: International 1924
- Prime Meridian: Greenwich

In [4]:
output_excel_path = cwd / "kaitak_gi.xlsx"

In [5]:
with open(gi_files[1]) as ags_file:
    ags3_data = ags_file.read()
ags3_db = ags_to_dfs(ags3_data)
brgi_db = ags3_to_brgi(ags3_db, crs)

AGS 3 data was read for Project GE/2005/03.57
This Ground Investigation data contains groups:
['PROJ', 'HOLE', 'CDIA', 'CORE', 'DETL', 'FLSH', 'GEOL', 'HDIA', 'PTIM', 'SAMP', '?LEGD', 'UNIT', 'ABBR', 'DICT']

Transforming AGS 3 groups to Bedrock tables...

'Project' table was created successfully.

'Location' table was created successfully.

'Sample' table was created successfully.



SchemaErrors: {
    "SCHEMA": {
        "WRONG_DATATYPE": [
            {
                "schema": "BaseInSitu",
                "column": "depth_to_top",
                "check": "dtype('float64')",
                "error": "expected series 'depth_to_top' to have type float64, got object"
            }
        ]
    }
}

In [ ]:
brgi_db = {}
for gi_file in gi_files:
    with open(gi_file) as ags_file:
        ags3_data = ags_file.read()
    ags3_db = ags_to_dfs(ags3_data)
    brgi_db = ags3_to_brgi(ags3_db, crs, brgi_db)

In [6]:
ags3_db_raw = ags_to_dfs(ags3_data)

AGS 3 data was read for Project GE/2005/03.57
This Ground Investigation data contains groups:
['PROJ', 'HOLE', 'CDIA', 'CORE', 'DETL', 'FLSH', 'GEOL', 'HDIA', 'PTIM', 'SAMP', '?LEGD', 'UNIT', 'ABBR', 'DICT']



In [7]:
ags3_db = ags3_db_raw.copy()
brgi_db = ags3_to_brgi(ags3_db, crs)

'Project' table was created successfully.

'Location' table was created successfully.

'Sample' table was created successfully.

'InSitu_CDIA' table was created successfully.

'InSitu_CORE' table was created successfully.

'InSitu_DETL' table was created successfully.

'InSitu_FLSH' table was created successfully.

'InSitu_GEOL' table was created successfully.

'InSitu_HDIA' table was created successfully.

'InSitu_PTIM' table was created successfully.



In [7]:
# Make a copy of the data before transforming
ags3_db = ags3_db_raw.copy()

ags_version = ags3_db["ags_version"]
del ags3_db["ags_version"]

# Instantiate Bedrock dictionary of pd.DataFrames
brgi_db = {}

# Project
brgi_db["Project"] = ags_proj_to_brgi_project(ags3_db["PROJ"], crs)
project_uid = brgi_db["Project"]["project_uid"].item()
del ags3_db["PROJ"]
print("'Project' table was created successfully.\n")

# Locations
if "HOLE" in ags3_db.keys():
    brgi_db["Location"] = ags3_hole_to_brgi_location(ags3_db["HOLE"], project_uid)
    del ags3_db["HOLE"]
    print("'Location' table was created successfully.\n")
else:
    print(
        "Your AGS 3 data doesn't contain a HOLE group, i.e. Ground Investigation locations."
    )

# Samples
if "SAMP" in ags3_db.keys():
    check_ags3_hole_ids_exist(brgi_db["Location"], ags3_db["SAMP"])
    ags3_db["SAMP"] = generate_sample_ids_for_ags3(ags3_db["SAMP"])
    brgi_db["Sample"] = ags3_samp_to_brgi_sample(ags3_db["SAMP"], project_uid)
    del ags3_db["SAMP"]
    print("'Sample' table was created successfully.\n")
else:
    print("Your AGS 3 data doesn't contain a SAMP group, i.e. samples.")

# The rest of the tables: 1. Lab Tests 2. In-Situ Tests 3. Other tables
for group, group_df in ags3_db.items():
    if "SAMP_REF" in ags3_db[group].columns:
        print(f"Project {project_uid} has lab test data: {group}.")
        brgi_db[group] = group_df
    elif "HOLE_ID" in ags3_db[group].columns:
        check_ags3_hole_ids_exist(brgi_db["Location"], group_df)
        brgi_db[f"InSitu_{group}"] = ags3_in_situ_to_brgi_in_situ(
            group, group_df, project_uid
        )
        print(f"'InSitu_{group}' table was created successfully.\n")
    else:
        brgi_db[group] = ags3_db[group]

'Project' table was created successfully.

'Location' table was created successfully.

'Sample' table was created successfully.

'InSitu_CDIA' table was created successfully.

'InSitu_CORE' table was created successfully.

'InSitu_DETL' table was created successfully.

'InSitu_FLSH' table was created successfully.

'InSitu_GEOL' table was created successfully.

'InSitu_HDIA' table was created successfully.

'InSitu_PTIM' table was created successfully.



In [10]:
brgi_db["InSitu_CDIA"]

,HOLE_ID,CDIA_CDEP,CDIA_HOLE,CDIA_REM,project_uid,location_uid,depth_to_top,depth_to_base
0,B1,6.00,140,None,GE/2005/03.57,B1_GE/2005/03.57,None,None
1,B10,6.00,140,None,GE/2005/03.57,B10_GE/2005/03.57,None,None
2,B11,6.00,140,None,GE/2005/03.57,B11_GE/2005/03.57,None,None
3,B12,6.00,140,None,GE/2005/03.57,B12_GE/2005/03.57,None,None
4,B13,6.00,140,None,GE/2005/03.57,B13_GE/2005/03.57,None,None
5,B14,6.00,140,None,GE/2005/03.57,B14_GE/2005/03.57,None,None
6,B15,6.00,140,None,GE/2005/03.57,B15_GE/2005/03.57,None,None
7,B16,6.00,140,None,GE/2005/03.57,B16_GE/2005/03.57,None,None
8,B17,6.00,140,None,GE/2005/03.57,B17_GE/2005/03.57,None,None
9,B18,6.00,140,None,GE/2005/03.57,B18_GE/2005/03.57,None,None


In [16]:
# Validate raw AGS 3 data with pandera schemas.
# Use try-catch to avoid crashing the notebook, while still displaying the errors in the data.

try:
    Ags3HOLE.validate(ags3_db_raw["HOLE"])
except Exception as e:
    print(e)

try:
    BaseSAMP.validate(ags3_db_raw["SAMP"])
except Exception as e:
    print(e)

try:
    Ags3GEOL.validate(ags3_db_raw["GEOL"])
except Exception as e:
    print(e)

try:
    Ags3ISPT.validate(ags3_db_raw["ISPT"])
except Exception as e:
    print(e)

try:
    Ags3CORE.validate(ags3_db_raw["CORE"])
except Exception as e:
    print(e)

try:
    Ags3WETH.validate(ags3_db_raw["WETH"])
except Exception as e:
    print(e)

'ISPT'
Error while coercing 'CORE_SREC' to type int64: Could not coerce <class 'pandas.core.series.Series'> data_container into type int64:
   index failure_case
0      0         None
1      1         None
2      2         None
3      3         None
4      4         None
5      5         None
6      6         None
'WETH'


In [11]:
write_gi_dfs_to_excel(ags3_db, output_excel_path)

Excel sheet names cannot contain [':', '/', '\\', '?', '*', '[', ']']. Replaced '?LEGD' with '_LEGD'
Ground Investigation data has been written to 'c:\Users\joost\ReposWindows\bedrock-gi\sandbox\kaitak_gi.xlsx'.


# Misc

In [ ]:
def prepend_column(df: pd.DataFrame, column_name: str, values) -> pd.DataFrame:
    """Make sure that pd.DataFrame.insert() doesn't cause errors when the column already exists.

    Args:
        df (pd.DataFrame): _description_
        column_name (str): _description_
        values (_type_): _description_

    Returns:
        pd.DataFrame: _description_
    """
    if column_name in df.columns:
        df.drop(columns=column_name, inplace=True)
    df.insert(loc=0, column=column_name, value=values)
    return df

# Speckle Stuff

In [ ]:
import datetime as dt

from dotenv import load_dotenv
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Line, Point
from specklepy.transports.server import ServerTransport


def to_speckle():
    load_dotenv()
    stream_id = "7fbe8ed384"
    hole_table_path = "data/1_split2/HOLE.csv"
    df = pd.read_csv(hole_table_path, index_col=0)
    client, stream_id = get_stream(stream_id)

    # next create a server transport - this is the vehicle through which you will send and receive
    transport = ServerTransport(client=client, stream_id=stream_id)

    hash = create_hash(df, transport)

    commit_hash(hash, client, stream_id)


def get_stream(stream_id):
    # Authenticate with Speckle server
    speckle_server = "app.speckle.systems"
    speckle_token = os.environ["speckle_token"]
    client = SpeckleClient(host=speckle_server)
    account = get_default_account()

    client.authenticate_with_token(speckle_token)

    # create a new stream. this returns the stream id
    if not stream_id:
        stream_id = client.stream.create(name="a shiny new stream")

    # use that stream id to get the stream from the server
    new_stream = client.stream.get(id=stream_id)
    return client, stream_id


def create_hash(df, transport):
    newObj = Base()
    for i, row in df.iterrows():
        x = row["HOLE_NATE"]
        y = row["HOLE_NATN"]
        z_top = row["HOLE_GL"]
        z_bot = z_top - row["HOLE_FDEP"]

        # GisPointElement, GisLineElement lijken niet te werken
        p1 = Point(x=x, y=y, z=z_top)
        p2 = Point(x=x, y=y, z=z_bot)
        line = Line(start=p1, end=p2)
        relevant_cols = [
            "HOLE_ID",
            "HOLE_TYPE",
            "HOLE_STAR",
            "HOLE_LOG",
            "?HOLE_DLOG",
            "?HOLE_CHEK",
            "?HOLE_DCHK",
            "HOLE_REM",
            "?HOLE_FLSH",
            "HOLE_ENDD",
            "HOLE_BACD",
            "HOLE_CREW",
            "HOLE_INCL",
            "HOLE_EXC",
        ]

        for col in relevant_cols:
            line[col] = row[col]

        newObj[f"myline{i}"] = line
        break

    # this serialises the block and sends it to the transport
    hash = operations.send(base=newObj, transports=[transport])
    return hash


def commit_hash(hash, client, stream_id):
    # you can now create a commit on your stream with this object
    commid_id = client.commit.create(
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


if __name__ == "__main__":
    to_speckle()
